In [26]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
%matplotlib inline
import pandas as pd

In [2]:
emotions = np.load('./data/labeled_emotion_josh_runzhi_adam.npz',encoding='latin1')
images_train = emotions['train']
labels_train = emotions['labels']
images_valid = emotions['valid']
labels_valid = emotions['labels_valid']

In [3]:
print('dimensions:')
print('training input:',images_train.shape,' output:',labels_train.shape)

dimensions:
training input: (8186,)  output: (8186,)


In [4]:
size = 64, 64
temp_train = []
for image in images_train:
    pic = Image.fromarray(image.astype('uint8'), 'RGB')
    pic = pic.resize(size)
    arr = np.array(pic,dtype='f')
    temp_train.append(arr)

In [5]:
size = 64, 64
temp_valid = []
for image in images_valid:
    pic = Image.fromarray(image.astype('uint8'), 'RGB')
    pic = pic.resize(size)
    arr = np.array(pic,dtype='f')
    temp_valid.append(arr)

In [6]:
imshape = 64*64*3
outputShape = 7

images_train = scale(np.asarray(temp_train).reshape(-1,64*64*3).astype('float64'),axis=1)
images_valid = scale(np.asarray(temp_valid).reshape(-1,64*64*3).astype('float64'),axis=1)

#reshape
# images_train = smaller_images_train.reshape(-1, imshape)

#one-hot encoding
labels_train = OneHotEncoder(sparse=False).fit_transform(labels_train.reshape(-1,1))

# images_valid = smaller_images_valid.reshape(-1, imshape)
#one-hot encoding
labels_valid = OneHotEncoder(sparse=False).fit_transform(labels_valid.reshape(-1,1))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_4x4(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1],
                        strides=[1, 4, 4, 1], padding='SAME')

In [7]:
x = tf.placeholder(tf.float32, shape=[None, imshape])
y_ = tf.placeholder(tf.float32, shape=[None, outputShape])

#First Convolutional Layer
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,64,64,3])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#Second Convolutional Layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#Densely Connected Layer
W_fc1 = weight_variable([16 * 16 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#drop_out
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#readout_layer
W_fc2 = weight_variable([1024, 7])
b_fc2 = bias_variable([7])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [16]:
N = len(images_train)
BS = 200
minibatchs = int(np.ceil(N/BS))
#i = epchos
for epoch in range(50):
    #scramble the indexes:
    ix = np.random.permutation(N)
    input_train = images_train[ix,:]
    input_label = labels_train[ix]
    for i in range(minibatchs):
        x_batch = images_train[i*BS:min(N, BS*(i+1))]
        y_batch = labels_train[i*BS:min(N, BS*(i+1))]
        train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: 0.5})

    if epoch%1 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:images_train, y_: labels_train, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(epoch, train_accuracy))
#         saver.save(sess, 'emotion_iter_'+str(epoch))
    
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: images_valid, y_: labels_valid, keep_prob: 1.0}))

step 0, training accuracy 0.23247
step 1, training accuracy 0.295627
step 2, training accuracy 0.333618
step 3, training accuracy 0.36428
step 4, training accuracy 0.406181
step 5, training accuracy 0.44173
step 6, training accuracy 0.490716
step 7, training accuracy 0.539335
step 8, training accuracy 0.57012
step 9, training accuracy 0.614952
step 10, training accuracy 0.659296
step 11, training accuracy 0.711459
step 12, training accuracy 0.745175
step 13, training accuracy 0.785976
step 14, training accuracy 0.810652
step 15, training accuracy 0.838627
step 16, training accuracy 0.860982
step 17, training accuracy 0.879184
step 18, training accuracy 0.891644
step 19, training accuracy 0.903372
step 20, training accuracy 0.91461
step 21, training accuracy 0.920596
step 22, training accuracy 0.929269
step 23, training accuracy 0.936966
step 24, training accuracy 0.942951
step 25, training accuracy 0.950037
step 26, training accuracy 0.95419
step 27, training accuracy 0.958588
step 28,

KeyboardInterrupt: 

In [17]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: images_valid, y_: labels_valid, keep_prob: 1.0}))

test accuracy 0.937653


In [18]:
p_valid = sess.run(y_conv, feed_dict={x: images_valid, keep_prob: 1.0})

In [19]:
label = np.array([0,1,2,3,4,5,6])

In [20]:
result = label[p_valid.argmax(axis=1)]

In [21]:
test = np.vstack((result,emotions['labels_valid']))

In [22]:
result2 = pd.DataFrame(test.T)

In [23]:
confusion_matrix(emotions['labels_valid'],result)

array([[270,   2,   7,   3,   7,   7,   0],
       [  1, 266,   2,   3,   4,   0,   0],
       [  3,   3, 277,  11,   6,   4,   0],
       [  1,   1,   2, 396,   4,   0,   2],
       [  1,   1,   5,   4, 215,   1,   5],
       [  3,   0,   5,   1,   5, 228,   1],
       [  0,   4,   1,   6,  11,   0, 258]])

In [15]:
labels = ['angry','happy','sad','neutral','fear','disgust','surprise']

In [51]:
def format_image(image):
    size = 64, 64
    pic = Image.fromarray(image.astype('uint8'), 'RGB')
    pic = pic.resize(size)
    arr = np.array(pic,dtype='f')
    arr = scale(arr.reshape(-1,64*64*3).astype('float64'),axis=1)
    return arr

In [77]:
path = './data/training_images_take2/'
files = os.listdir(path).copy()
testset=[]
for file in files:
    curIm = np.array(Image.open(path+file))
    testset.append(format_image(curIm))

In [80]:
testset=np.array(testset)
testset=testset.reshape(testset.shape[0],testset.shape[2])

In [81]:
testset.shape

(473, 12288)

In [82]:
p_test = sess.run(y_conv, feed_dict={x: format_file, keep_prob: 1.0})

In [83]:
result = label[p_test.argmax(axis=1)]